In [ ]:
# tryCatch({
#   source("here::here("/scratch_isilon/groups/singlecell/pnieto/scripts/r_utils/utils.R")")
# }, error = function(e) {
#   source("here::here("S:/scripts/r_utils/utils.R")")
# })

source("here::here("S:/scripts/r_utils/utils.R")")


In [ ]:
# create output folder if it doesn't exist yet
if (!file.exists(params$out_dir)) {
  dir.create(params$out_dir, recursive = TRUE)
}


In [ ]:
cat("All parameters passed to render this file:", "\n")
t <- lapply(1:length(params), function(x){
  cat(paste0(names(params[x]), ": " , params[[x]]), "\n")
})


`r glue::glue("# {params$proj_name}")`


There are three important variables for the QC filtering:


- *nFeature_RNA* is the number of genes expressed in each cell


- *nCount_RNA* is the total number of RNA molecules detected in a cell (number of UMIs)


- *percent.mt* is the percentage of expressed RNA that corresponds to mitochondrial genes


- *doublet_score* is the score that a cell might be a doublet, instead of a single cell


Low number of genes or RNA molecules for a cell indicates that it may be dead or an empty droplet. On the contrary, cells with high number of genes or molecules indicate that the "cell" may in fact be a doublet (or multiplet). Cells with high mitochondrial content may also be dead or broken cells as the mitochondrial material is the last to degrade.


By removing outliers from these groups we expect to remove most bad quality cells from the downstream analysis. However, we won't be very stringent here, as we can further remove bad quality cells during downstream analysis.

In [ ]:

library <- list.dirs(path = glue::glue("{params$proj_dir}/jobs"), full.names = FALSE, recursive = FALSE)

obj_list <- purrr::map(library, function(lib){
  rna <- glue::glue("{params$proj_dir}/jobs/{lib}/{lib}/outs/per_sample_outs/{lib}/count/sample_filtered_feature_bc_matrix/") 
  print(rna)
  tcr <- glue::glue("{params$proj_dir}/jobs/{lib}/{lib}/outs/per_sample_outs/{lib}/vdj_t/")
  print(tcr)
  bcr <- glue::glue("{params$proj_dir}/jobs/{lib}/{lib}/outs/per_sample_outs/{lib}/vdj_b/")
  print(bcr)
  load_seurat(gex_dir = rna, tcr_dir = NULL, bcr_dir = NULL, project = params$proj_name, sample = lib, sp = "H")
})
saveRDS(obj_list, glue::glue("{params$out_dir}/{params$proj_name}_raw_obj_list.rds"))


In [ ]:
obj_list <- readRDS(glue::glue("{params$out_dir}/{params$proj_name}_raw_obj_list.rds"))


In [ ]:
# merge objects before QC (only if length of list is greater than one)
if (length(obj_list) > 1) {data <- merge(obj_list[[1]], obj_list[2:length(obj_list)])} else {data <- obj_list[[1]]
}
print(data)
rm(obj_list)
gc()


## General QC visualization


In [ ]:
Idents(data) <- "sample"
qcplots(data)


## Library Size


In [ ]:
lib_size_hist1 <- data@meta.data %>%
  ggplot(aes(nCount_RNA)) +
    geom_histogram(bins = 100) +
    labs(x = "Library Size (log10(total UMI))", y = "Number of Cells") +
    geom_vline(xintercept = params$min_lib_size, color = "red") +
    geom_vline(xintercept = params$max_lib_size, color = "red") +
    scale_x_log10() +
    theme_pubr()
lib_size_hist2 <- data@meta.data %>%
  ggplot(aes(nCount_RNA)) +
    geom_histogram(bins = 100) +
    scale_x_continuous(limits = c(min(data$nCount_RNA), 100000)) +
    labs(x = "Library Size (total UMI)", y = "Number of Cells") +
    geom_vline(xintercept = params$min_lib_size, color = "red") +
    geom_vline(xintercept = params$max_lib_size, color = "red") +
    theme_pubr()
ggarrange(plotlist = list(lib_size_hist1, lib_size_hist2), ncol = 2)


## Number of detected genes


In [ ]:
n_genes_hist1 <- data@meta.data %>%
  ggplot(aes(nFeature_RNA)) +
    geom_histogram(bins = 100) +
    labs(x = "Number of Detected Genes (log10)", y = "Number of Cells") +
    geom_vline(xintercept = params$min_n_genes, color = "red") +
    geom_vline(xintercept = params$max_n_genes, color = "red") +
    scale_x_log10() +
    theme_pubr()
n_genes_hist2 <- data@meta.data %>%
  ggplot(aes(nFeature_RNA)) +
    geom_histogram(bins = 100) +
    #scale_x_continuous(limits = c(0, 1000)) +
    geom_vline(xintercept = params$min_n_genes, color = "red") +
    geom_vline(xintercept = params$max_n_genes, color = "red") +
    labs(x = "Number of Detected Genes", y = "Number of Cells") +
    theme_pubr()
ggarrange(plotlist = list(n_genes_hist1, n_genes_hist2), ncol = 2)


Number of detected genes across samples:


In [ ]:
n_genes_vs_samples <- VlnPlot(
  data,
  features = "nFeature_RNA",
  pt.size = 0,
  #cols = rep("darkgray", length(unique(data$patient))),
  group.by = "sample"
)
n_genes_vs_samples +
  labs(title = "", x = "", y = "Number of Detected Genes") +
  geom_hline(yintercept = params$min_n_genes, color = "red") +
  geom_hline(yintercept = params$max_n_genes, color = "red") +
  #scale_y_log10() +
  theme(legend.position = "none",
        axis.title = element_text(size = 13))


## Fraction of mitochondrial content


In [ ]:
pct_mt_hist <- data@meta.data %>%
  ggplot(aes(percent.mt)) +
    geom_histogram(bins = 10) +
    geom_vline(xintercept = params$max_pct_mt, color = "red") +
    #scale_x_continuous(limits = c(0, 100)) +
    #scale_y_continuous(limits = c(0, 1000)) +
    labs(x = "% Mitochondrial Expression", y = "Number of Cells") +
    
    theme_pubr()
pct_mt_hist


In [ ]:
mt_pct_vs_samples <- VlnPlot(
  data,
  features = "percent.mt",
  pt.size = 0,
  #cols = rep("darkgray", length(unique(data$patient))),
  group.by = "sample",
  y.max = 100
)
mt_pct_vs_samples +
  labs(title = "", x = "", y = "% of MT genes") +
  geom_hline(yintercept = params$max_pct_mt, color = "red") +
  #scale_y_log10() +
  theme(legend.position = "none",
        axis.title = element_text(size = 13))


## Joint QC metrics


It is important to assess how these variables covary, since metabolically active cells might also have a high mitochondrial expression:


In [ ]:
# number of detected genes VS library size
n_genes_vs_lib_size <- FeatureScatter(
  data,
  feature1 = "nCount_RNA",
  feature2 = "nFeature_RNA",
  pt.size = 0.15,
  cols = rep("black", length(levels(Idents(data))))
)
n_genes_vs_lib_size <- n_genes_vs_lib_size +
  labs(x = "Library Size (total UMI)", y = "Number of Detected Genes") +
  theme(legend.position = "none", plot.title = element_blank())
n_genes_vs_lib_size +
  geom_vline(xintercept = params$min_lib_size, linetype = "dashed", color = "red") +
  geom_vline(xintercept = params$max_lib_size, linetype = "dashed", color = "red") +
  geom_hline(yintercept = params$min_n_genes, linetype = "dashed", color = "red") +
  geom_hline(yintercept = params$max_n_genes, linetype = "dashed", color = "red")


In [ ]:
# % mitochondrial expression VS library size
pct_mt_vs_lib_size <- FeatureScatter(
  data,
  feature1 = "nCount_RNA",
  feature2 = "percent.mt",
  pt.size = 0.15,
  cols = rep("black", length(levels(Idents(data))))
)
pct_mt_vs_lib_size <- pct_mt_vs_lib_size +
  labs(x = "Library Size (total UMI)", y = "% Mitochondrial Expression") +
  theme(legend.position = "none", plot.title = element_blank())
pct_mt_vs_lib_size +
  geom_vline(xintercept = params$min_lib_size, linetype = "dashed", color = "red") +
  geom_vline(xintercept = params$max_lib_size, linetype = "dashed", color = "red") +
  geom_hline(yintercept = params$max_pct_mt, linetype = "dashed", color = "red")


In [ ]:
mid <- 3

p  = ggplot(data@meta.data, aes(x = nCount_RNA, y = nFeature_RNA)) + 
  geom_point(aes(color = percent.mt), size = 1) +
  scale_color_gradient2(midpoint = mid, low = "black", mid = grDevices::adjustcolor("grey", alpha.f = 0.2), high ="red") +
  scale_y_continuous(trans = "log10") + 
  scale_x_continuous(trans = "log10") +
  theme_cowplot(12) + 
  geom_vline(xintercept = c(params$min_lib_size,params$max_lib_size), linetype = 3) +
  geom_hline(yintercept = c(params$min_n_genes,params$max_n_genes), linetype = 3)

ggMarginal(p, type = "densigram", size = 10)


In [ ]:
mid <- 5

ggplot(data@meta.data, aes(x = nCount_RNA, y = nFeature_RNA)) + 
  geom_point(aes(color = percent.mt), size = 1) +
  scale_color_gradient2(midpoint = mid, low = "black", mid = grDevices::adjustcolor("white", alpha.f = 0.2), high ="red") +
  scale_y_continuous(trans = "log10") + 
  scale_x_continuous(trans = "log10") + 
  theme_cowplot(12) + 
  facet_wrap(~sample, ncol = 2) + 
  geom_vline(xintercept = c(params$min_lib_size, params$max_lib_size), linetype = 3) + 
  geom_hline(yintercept = c(params$min_n_genes, params$max_n_genes), linetype = 3)


## Scrublet


We run [scrublet](https://www.sciencedirect.com/science/article/pii/S2405471218304745) on each of the samples but for the moment we are not going to filter out any cells based on doublet score.


In [ ]:
mid <- 0.05

p  = ggplot(data@meta.data, aes(x = nCount_RNA, y = nFeature_RNA)) + 
  geom_point(aes(color =doublet_score), size = 1) +
  scale_color_gradient2(midpoint = mid, low = "black", mid = grDevices::adjustcolor("grey", alpha.f = 0.2), high ="red") +
  scale_y_continuous(trans = "log10") + 
  scale_x_continuous(trans = "log10") +
  theme_cowplot(12) + 
  geom_vline(xintercept = c(params$min_lib_size,params$max_lib_size), linetype = 3) +
  geom_hline(yintercept = c(params$min_n_genes,params$max_n_genes), linetype = 3)

ggMarginal(p, type = "densigram", size = 10)


In [ ]:
mid <- 0.05

ggplot(data@meta.data, aes(x = nCount_RNA, y = nFeature_RNA)) + 
  geom_point(aes(color =doublet_score), size = 1) +
  scale_color_gradient2(midpoint = mid, low = "black", mid = grDevices::adjustcolor("white", alpha.f = 0.2), high ="red") +
  scale_y_continuous(trans = "log10") + 
  scale_x_continuous(trans = "log10") + 
  theme_cowplot(12) + 
  facet_wrap(~sample, ncol = 2) + 
  geom_vline(xintercept = c(params$min_lib_size, params$max_lib_size), linetype = 3) + 
  geom_hline(yintercept = c(params$min_n_genes, params$max_n_genes), linetype = 3)


## Filtering


- between `r params$min_lib_size` and `r params$max_lib_size` UMIs per cell


- between `r params$min_n_genes` and `r params$max_n_genes` genes


- less than `r params$max_pct_mt`% mitochondrial content


How many cells are we keeping?


In [ ]:
metadata_before_qc <- data@meta.data

is_low_quality <- 
  data$nCount_RNA < params$min_lib_size |
  data$nCount_RNA > params$max_lib_size |
  data$nFeature_RNA < params$min_n_genes |
  data$nFeature_RNA > params$max_n_genes |
  data$percent.mt > params$max_pct_mt
data$keep_cells <- !is_low_quality
Idents(data) <- "keep_cells"
table(data$keep_cells)


In [ ]:
purrr::map(unique(data$sample), function(s){
  saveRDS(subset(data, sample == s), glue("{params$out_dir}/{params$proj_name}_{s}_raw.rds"))
})
saveRDS(data, glue("{params$out_dir}/{params$proj_name}_all_raw.rds"))


In [ ]:
data <- subset(data, subset = keep_cells == TRUE)
metadata_after_qc <- data@meta.data


## QC summary table


Comparison before and after filtering


In [ ]:
qc_before <- metadata_before_qc %>%
  group_by(sample) %>% 
  summarise(num_cells_before_qc = n())
qc_after <- metadata_after_qc %>%
  group_by(sample) %>%
  summarise(
    num_cells_after_qc = n(),
    average_library_size = mean(nCount_RNA),
    average_num_detected_genes = mean(nFeature_RNA),
    average_mitochondrial_fraction = mean(percent.mt)
  )
qc_table <- left_join(qc_before, qc_after, by = "sample")
flextable::flextable(qc_table) %>% 
  flextable::autofit()


In [ ]:
purrr::map(unique(data$sample), function(s){
  saveRDS(subset(data, sample == s), glue("{params$out_dir}/{params$proj_name}_{s}_filtered.rds"))
})
saveRDS(data, glue("{params$out_dir}/{params$proj_name}_all_filtered.rds"))


***


<a href="#top">Back to top</a>


## Session Info


In [ ]:
sessionInfo()


<a href="#top">Back to top</a>
